In [1]:
"""
This is just a sample code for building a CNNClassifier in PyTorch
It is not using any atual data for training
"""

import torch
from torch import nn

    
def calc_conv_out_seq_len(seq_len, kernel_len, stride=1, dilation=1, padding=0):
    return (
        1 + (seq_len + 2 * padding - dilation * (kernel_len - 1) - 1) //stride
        )


class CNNTextClassifier(nn.Module):
    def __init__(self, embeddings):
        super().__init__()

        self.seq_len = 40
        self.vocab_size = 10000
        self.embedding_size = 50
        self.out_channels = 5
        self.kernel_lenghts = [2,3,4,5,6]
        self.strid = 1
        self.dropout = nn.Droput(0)
        self.pool_strid = self.strid
        self.conv_out_seq_len = calc_out_seq_len(
                                        self_len=self.seq_len,
                                        kernel_lengths=self.kernel_lengths,
                                        strid=self.stride
                                        )

        self.embed = nn.Embedding(
            self.vocab_size,
            self.embdedding_size,
            padding_idx=0)
        state = self.embed.state_dict()
        state['weight'] = embeddings
        self.embed.load_state_dict(state)

        self.convolvers = []
        self.poolers = []
        total_out_len = 0

        for i, kernel_len in enumerate(self.kernel_lengths):
            self.convolvers.append(
                nn.Conv1d(in_channels=self.embedding_size,
                          out_channels=self.out_channels,
                          kernel_size=kernel_len,
                          stride=self.stride)
                )

            print(f'conv[{i}].weight.shape: {self.convolvers[-1].weight.shape}') 
            conv_output_len = calc_conv_out_seq_len(seq_len=self.seq_len, kernel_len=kernel_len, stride=self.stride)
            print(f'conv_output_len: {conv_output_len}')

            self.poolers.append(
                nn.MaxPool1d(kernel_size=conv_output_len, 
                             stride=self.stride)
                )
            total_out_len += calc_conv_out_seq_len( seq_len=conv_output_len, kernel_len=conv_output_len, stride=self.stride)
            print(f'total_out_len: {total_out_len}')
            print(f'poolers[{i}]: {self.poolers[-1]}')


        print(f'total_out_len: {total_out_len}') 
        self.linear_layer = nn.Linear(self.out_channels * total_out_len, 1)
        print(f'linear_layer: {self.linear_layer}')

    def forward(self, x):
        embedded = self.embed(x).permute(0, 2, 1)  # Embedding layer + permutation for Conv1d
        pooled_outputs = []

        for convolver, pooler in zip(self.convolvers, self.poolers):
            conv_output = convolver(embedded)
            pooled_output = pooler(conv_output)
            pooled_outputs.append(pooled_output)

        combined_output = torch.cat(pooled_outputs, dim=1)  # Concatenate pooled outputs along channel dimension
        flattened_output = combined_output.view(-1, self.linear_input_size)  # Flatten the pooled outputs
        final_output = self.linear_layer(flattened_output)  # Linear layer

        return torch.sigmoid(final_output)  # Sigmoid activation for binary classification

In [ ]:
# Create an instance of the model
model = CNNTextClassifier(embeddings)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary cross-entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for inputs, labels in train_loader:  # Iterate over batches of training data
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels.float())  # Calculate the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the parameters

    # Validation loop (optional)
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:  # Iterate over batches of validation data
            val_outputs = model(val_inputs)  # Forward pass
            val_loss = criterion(val_outputs, val_labels.float())  # Calculate the loss

    # Print training and validation metrics (optional)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')
